# Homework 5

In [34]:
from landlab.components import LinearDiffuser
from landlab.plot import imshow_grid
from landlab import RasterModelGrid, CLOSED_BOUNDARY, FIXED_VALUE_BOUNDARY
from matplotlib.pyplot import figure, show, plot, xlabel, ylabel, title
import numpy as np

from landlab.components import FlowRouter, FastscapeEroder
from landlab import load_params

In [35]:
## Define parameters
####################

# Import parameters from a txt file
input_file = './coupled_params.txt'
inputs = load_params(input_file) # load the data into a dictionary

# Write paramter from txt file
nrows = inputs['nrows'] # Nombre de ligne
ncols = inputs['ncols'] # Nombre de colonne
dx = inputs['dx'] # Espacement dans les deux directions [m]
uplift_rate = inputs['uplift_rate'] #[m/s]
total_t = inputs['total_time']
dt = inputs['dt']

Area = nrows*ncols*dx # [m2]
Lc = np.sqrt(Area)

#Set parameters for modelisation
nt = int(total_t // dt) #this is how many loops we'll need
uplift_per_step = uplift_rate * dt

# illustrate what the MPD looks like:
# print(inputs)
print(Area,Lc)

ValueError: parsing of parameter file did not produce a dict-like object

In [23]:
## Create and define grid
#########################

# Create grid
mg = RasterModelGrid((nrows, ncols), dx)

# Create matrice of eleveation z
z = mg.add_zeros('node', 'topographic__elevation')

# add some roughness, as this lets "natural" channel planforms arise
initial_roughness = np.random.rand(z.size)/100000.
z += initial_roughness

# Set boundaries, top and bottom with fixed value, right and left closed
for edge in (mg.nodes_at_left_edge, mg.nodes_at_right_edge):
    mg.status_at_node[edge] = CLOSED_BOUNDARY
for edge in (mg.nodes_at_top_edge, mg.nodes_at_bottom_edge):
    mg.status_at_node[edge] = FIXED_VALUE_BOUNDARY

In [9]:
## Implement a run of the model
###############################

fr = FlowRouter(mg, **inputs)
sp = FastscapeEroder(mg, **inputs)
lin_diffuse = LinearDiffuser(mg, **inputs)
    
# LinearDiffuser(grid, linear_diffusivity=None, method='simple')

In [ ]:
# from landlab import RasterModelGrid
# import numpy as np
# g = RasterModelGrid((9, 9), 1.)
# z = mg.add_zeros('node', 'topographic__elevation')
z.reshape((9, 9))[4, 4] = 1.
mg.set_closed_boundaries_at_grid_edges(True, True, True, True)
ld = LinearDiffuser(mg, linear_diffusivity=1.)
for i in range(1):
...     ld.run_one_step(1.)
np.isclose(z[mg.core_nodes].sum(), 1.)
True
mg2 = RasterModelGrid((5, 30), 1.)
z2 = mg2.add_zeros('node', 'topographic__elevation')
z2.reshape((5, 30))[2, 8] = 1.
z2.reshape((5, 30))[2, 22] = 1.
mg2.set_closed_boundaries_at_grid_edges(True, True, True, True)
kd = mg2.node_x/mg2.node_x.mean()
ld2 = LinearDiffuser(mg2, linear_diffusivity=kd)
for i in range(10):
...     ld2.run_one_step(0.1)
z2[mg2.core_nodes].sum() == 2.
True
z2.reshape((5, 30))[2, 8] > z2.reshape((5, 30))[2, 22]
True


In [7]:
figure('Title')
imshow_grid(mg, 'topographic__elevation', grid_units=['km','km'], var_name='Elevation (km)')

In [29]:
LinearDiffuser?